In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Lasso
from sklearn.metrics import root_mean_squared_error

def read_dataframe(path):
    df = pd.read_parquet(path)
    df["duration"] = (df.lpep_dropoff_datetime - df.lpep_pickup_datetime).dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    df[["PULocationID", "DOLocationID"]] = df[["PULocationID", "DOLocationID"]].astype(str)
    return df

df_train = read_dataframe("../data/green_tripdata_2025-01.parquet")
df_val = read_dataframe("../data/green_tripdata_2025-02.parquet")

df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

categorical = ["PU_DO"]
numerical = ["trip_distance"]

dv = DictVectorizer()
X_train = dv.fit_transform(df_train[categorical + numerical].to_dict(orient="records"))
X_val = dv.transform(df_val[categorical + numerical].to_dict(orient="records"))

y_train = df_train["duration"].values
y_val = df_val["duration"].values

# **Método 1 para loggear un Model en el experimento**

In [3]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("class-nyc-taxi-experiment")

# Inicia un run que se llame tal
with mlflow.start_run(run_name="lasso_alpha_0.1"):
    # Definir parametro alpha
    alpha = 0.1
    # Logearlo
    mlflow.log_param("alpha", alpha)

    # Definir modelo y realizar predicciones
    model = Lasso(alpha=alpha)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)

    # Métrica Eval
    rmse = root_mean_squared_error(y_val, y_pred)

    # Logear la métrica
    mlflow.log_metric("rmse", rmse)

    # Logear el modelo se sklear usando sklearn [cambair por el que uses tf, keras etc]
    mlflow.sklearn.log_model(model, "model")

    print(f"✅ Run finalizado. RMSE = {rmse:.4f}")

2025/10/07 21:40:39 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/07 21:40:39 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

✅ Run finalizado. RMSE = 8.9926


# **Model Registry**

In [2]:
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment(experiment_name="nyc-taxi-model-registry-example")

2025/10/14 21:21:16 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/14 21:21:16 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025/10/14 21:21:17 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-model-registry-example' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///c:/Users/diana/OneDrive/Documentos/PROYECTO_CIENCIA_DATOS/REPOSITORIOS/nyc-taxi-predictions-2025/notebooks/mlruns/2', creation_time=1760498477335, experiment_id='2', last_update_time=1760498477335, lifecycle_stage='active', name='nyc-taxi-model-registry-example', tags={}>

In [3]:
mlflow.sklearn.autolog()

In [4]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2025-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2025-02")

In [5]:
models = [
    
    {"model": GradientBoostingRegressor,
     "params": {"n_estimators": 100, "learning_rate": 0.3, "max_depth": 25, "random_state": 42},
     },
    
    {"model": ExtraTreesRegressor,
     "params": {"n_estimators": 100, "max_depth": 15, "random_state": 42},
     },
    
    {"model": LinearSVR,
     "params": {"C": 1.0, "epsilon": 0}, 
     },

]

In [6]:
import pickle

In [7]:
with mlflow.start_run(run_name="Nested Runs"):
    for model in models:
        
        model_class = model["model"]
        model_name = model_class.__name__
        params = model["params"]
        
        with mlflow.start_run(run_name=model_name,nested=True):
            
            ml_model = model_class(**params)
           
            ml_model.fit(X_train, y_train)
    
            y_pred = ml_model.predict(X_val)
            
            rmse = root_mean_squared_error(y_val, y_pred)
            mlflow.log_metric("rmse", rmse)
            
            !mkdir models
            with open("models/preprocessor.b", "wb") as f_out:
                pickle.dump(dv, f_out)
                
            mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

2025/10/14 21:26:16 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/14 21:26:53 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Ya existe el subdirectorio o el archivo models.
c:\Users\diana\OneDrive\Documentos\PROYECTO_CIENCIA_DATOS\REPOSITORIOS\nyc-taxi-predictions-2025\.venv\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
2025/10/14 21:27:13 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Ya existe el subdirectorio o el archivo models.


# **Registrar Modelo con código: log_model**

In [8]:
from sklearn.ensemble import RandomForestRegressor


with mlflow.start_run(run_name="RandomForestRegressor"):
    ml_model = RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        random_state=42
    )
    
    ml_model.fit(X_train, y_train)
    
    mlflow.sklearn.log_model(
        sk_model=model, 
        artifact_path="model",
        registered_model_name="nyc-taxi-model"
    )
    
    y_pred = ml_model.predict(X_val)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    # !mkdir models
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

2025/10/14 21:45:06 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/14 21:45:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/14 21:45:11 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/10/14 21:45:28 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/14 21:45:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/10/14 21:45:29 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/14 21:45:29 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INF

# **Registrar Modelo con código: register_model**

In [10]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
2025/10/14 21:51:02 WARNING mlflow.tracking._model_registry.fluent: Run with id 00e52442bdb54d9b963ac540e9a9540a has no artifacts at artifact path 'model', registering model based on models:/m-6f71330053204169a69f8eb416a54f45 instead
Created version '3' of model 'nyc-taxi-model'.


# **Registrar Modelo con código: client.create_model_version**

In [2]:
from mlflow import MlflowClient

client = MlflowClient(tracking_uri="sqlite:///mlflow.db")

2025/10/16 20:30:17 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/16 20:30:17 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


In [3]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = client.create_model_version(
    name="nyc-taxi-model",
    source=run_uri,
    run_id=run_id
)

2025/10/16 20:31:13 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/16 20:31:13 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


# **Cambiar aliases con código: client.set_registered_model_alias**

In [4]:
# create "champion" alias for version 1 of model "example-model"
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="champion",
    version=4
)

In [5]:
# set the "challenger" alias to version 2
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="challenger",
    version=3
)

In [6]:
# Actualizar descripción de una versión
client.update_model_version(
    name="nyc-taxi-model",
    version=4,
    description="This model version is a scikit-learn xtra tree regressor containing 100 decision trees",
)

<ModelVersion: aliases=['champion'], creation_timestamp=1760668273324, current_stage='None', deployment_job_state=None, description=('This model version is a scikit-learn xtra tree regressor containing 100 '
 'decision trees'), last_updated_timestamp=1760669213439, metrics=None, model_id=None, name='nyc-taxi-model', params=None, run_id='6ee08f9df81147a3aa6c5a865e76277d', run_link=None, source='runs:/6ee08f9df81147a3aa6c5a865e76277d/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [7]:
# delete the alias
client.delete_registered_model_alias(
    name="nyc-taxi-model", 
    alias="challenger"
)

In [8]:
# get a model version by alias
client.get_model_version_by_alias(
    name="nyc-taxi-model",
    alias="champion"
)

<ModelVersion: aliases=['champion'], creation_timestamp=1760668273324, current_stage='None', deployment_job_state=None, description=('This model version is a scikit-learn xtra tree regressor containing 100 '
 'decision trees'), last_updated_timestamp=1760669213439, metrics=None, model_id=None, name='nyc-taxi-model', params=None, run_id='6ee08f9df81147a3aa6c5a865e76277d', run_link=None, source='runs:/6ee08f9df81147a3aa6c5a865e76277d/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

# **Obteniendo modelos del registro de modelos**

In [27]:
# Obtenerlo por versión de modelo:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
model_version = 1

model_uri = f"models:/{model_name}/{model_version}"

model = mlflow.pyfunc.load_model(
    model_uri=model_uri,
)

model.predict(X_val)

array([ 4.98661726, 24.1397534 , 26.9647779 , ..., 19.72875784,
       15.33514682, 22.07257903], shape=(44218,))

In [20]:
# Obtenerlo por alias
model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)

array([15.02976639, 15.02976639, 15.02976639, ..., 15.02976639,
       15.02976639, 15.02976639], shape=(44218,))

# **Comparación de versiones y selección del nuevo modelo champion**

In [28]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, alias, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}@{alias}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [30]:
df = read_dataframe("../data/green_tripdata_2025-03.parquet")

In [31]:
run_id = input("Ingrese el run_id")

client.download_artifacts(
    run_id=run_id, 
    path='preprocessor', 
    dst_path='.'
)

'c:\\Users\\diana\\OneDrive\\Documentos\\PROYECTO_CIENCIA_DATOS\\REPOSITORIOS\\nyc-taxi-predictions-2025\\notebooks\\preprocessor'

In [33]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [34]:
X_test = preprocess(df, dv)

In [35]:
target = "duration"
y_test = df[target].values

In [43]:
test_model(name="nyc-taxi-model", alias="champion", X_test=X_test, y_test=y_test)

{'rmse': 6.01147524270201}

In [41]:
test_model(name=model_name, alias="challenger", X_test=X_test, y_test=y_test)

{'rmse': 552.7034891640062}

In [ ]:
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="champion",
    version=100
)